## Загрузим нужные библиотеки

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import geopandas as gpd

In [16]:
from src.util import df_to_gdf, add_region, encode_region

In [17]:
DATA_DIR = "./data/"

In [50]:
df = pd.read_csv(DATA_DIR + "train_dataset_train.csv")

In [19]:
df = df_to_gdf(df)
regions = gpd.read_file(DATA_DIR + "boundaries/admin_level_4.shp", encoding='cp1251')
regions = regions[["name_en", "geometry"]]
add_region(df, regions)
encode_region(df)

In [21]:
df.head(3)

,id,area,nd_mean_2021-04-16,nd_mean_2021-04-19,nd_mean_2021-04-22,nd_mean_2021-04-26,nd_mean_2021-04-28,nd_mean_2021-05-02,nd_mean_2021-05-04,nd_mean_2021-05-07,...,nd_mean_2021-06-05,nd_mean_2021-06-10,nd_mean_2021-07-05,nd_mean_2021-08-13,nd_mean_2021-08-27,nd_mean_2021-05-08,nd_mean_2021-05-24,crop,geometry,region
0,3536,20,0.072846,0.261778,0.062981,0.104442,0.021096,0.052202,0.158723,0.000000,...,0.000000,0.000000,0.026784,0.126832,0.614770,0.008857,0.081498,3,"POLYGON ((42.49107 51.41404, 42.49111 51.41400...",13
1,3739,45,0.332928,0.287182,0.097138,0.152467,0.075484,0.000000,0.140326,0.163340,...,0.000000,0.000000,0.111148,0.174914,0.179612,0.113071,0.046997,4,"POLYGON ((50.25545 56.60539, 50.25552 56.60523...",11
2,1294,28,0.013679,0.007055,0.120804,0.037839,0.259125,0.000000,0.037446,0.506516,...,0.210496,0.130467,0.155225,0.090607,0.054127,0.007437,0.219614,2,"POLYGON ((37.99699 53.83590, 38.00572 53.83333...",12


## Выделим выборки

In [51]:
X = df.drop(["id",".geo", "crop"], axis = 1)
y = df[["crop"]]

In [52]:
X.head()

,area,nd_mean_2021-04-16,nd_mean_2021-04-19,nd_mean_2021-04-22,nd_mean_2021-04-26,nd_mean_2021-04-28,nd_mean_2021-05-02,nd_mean_2021-05-04,nd_mean_2021-05-07,nd_mean_2021-05-16,...,nd_mean_2021-05-19,nd_mean_2021-05-20,nd_mean_2021-06-04,nd_mean_2021-06-05,nd_mean_2021-06-10,nd_mean_2021-07-05,nd_mean_2021-08-13,nd_mean_2021-08-27,nd_mean_2021-05-08,nd_mean_2021-05-24
0,20,0.072846,0.261778,0.062981,0.104442,0.021096,0.052202,0.158723,0.000000,0.484102,...,0.425571,0.108999,0.118854,0.000000,0.000000,0.026784,0.126832,0.614770,0.008857,0.081498
1,45,0.332928,0.287182,0.097138,0.152467,0.075484,0.000000,0.140326,0.163340,0.519456,...,0.248935,0.134469,0.208268,0.000000,0.000000,0.111148,0.174914,0.179612,0.113071,0.046997
2,28,0.013679,0.007055,0.120804,0.037839,0.259125,0.000000,0.037446,0.506516,0.000000,...,0.127633,0.084467,0.000000,0.210496,0.130467,0.155225,0.090607,0.054127,0.007437,0.219614
3,19,0.105976,0.052500,0.001917,0.011531,0.042316,0.000000,0.000000,0.043604,0.510271,...,0.527583,0.000000,0.642226,0.132886,0.000000,0.570928,0.073492,0.378900,0.327677,0.586523
4,33,0.130885,0.063492,0.024416,0.000000,0.150671,0.046717,0.254284,0.101479,0.000000,...,0.000000,0.206158,0.099058,0.152003,0.073357,0.147842,0.157676,0.012048,0.054223,0.017539


## Обучение модели

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [69]:
clf = RandomForestClassifier(random_state=42, n_estimators=200)

In [70]:
clf.fit(X_train, y_train)

/tmp/ipykernel_5194/533357445.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, y_train)


RandomForestClassifier(n_estimators=200, random_state=42)

## Оценка точности

In [71]:
from sklearn.metrics import recall_score

pred = clf.predict(X_test)
print(clf,'\n',recall_score(y_test, pred, average="macro", zero_division=0))

RandomForestClassifier(n_estimators=200, random_state=42) 
 0.965580789035347


In [76]:
test_df = pd.read_csv(DATA_DIR + "test_dataset_test.csv")

In [77]:
X_submission = test_df.drop(["id",".geo"], axis = 1)
y_submission = clf.predict(X_submission)

In [79]:
submission_df = test_df.copy().set_index("id")
submission_df["crop"] = y_submission
submission_df = submission_df[["crop"]]
submission_df.head()

,crop
id,
611,3
6417,2
3352,3
4224,1
3102,6


In [80]:
submission_df.to_csv(DATA_DIR + "submission.csv")